# 简介
* 本教程将介绍如何使用 PaddleClas 套件导出推理模型并使用 WebAI.js 部署到网页前端

## 1. 同步代码
* 从 Github 或 Gitee 上克隆 PaddleClas 代码

In [ ]:
!git clone https://gitee.com/PaddlePaddle/PaddleClas --depth 1

## 2. 导出 Paddle 推理模型
* 推理脚本位于 PaddleClas/tools/export_model.py
* 更多更详细的使用方法可参考 PaddleClas 官方文档 PaddleClas/docs/zh_CN/inference_deployment/export_model.md
* 作为演示，所以使用官方文档中提供的示例模型进行模型导出

### 2.1 切换工作目录

In [ ]:
%cd PaddleClas

### 2.2 下载官方提供的预训练模型

In [ ]:
!wget -P ./cls_pretrain/ https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/legendary_models/ResNet50_vd_pretrained.pdparams

### 2.3 导出 Paddle 格式的推理模型
* 使用脚本时通过命令行参数指定模型的配置文件、预训练模型、保存目录和当前运行的设备类型

In [ ]:
!python tools/export_model.py \
    -c ./ppcls/configs/ImageNet/ResNet/ResNet50_vd.yaml \
    -o Global.pretrained_model=./cls_pretrain/ResNet50_vd_pretrained \
    -o Global.save_inference_dir=./deploy/models/class_ResNet50_vd_ImageNet_infer \
    -o Global.device=cpu

## 3. 转换为 ONNX 模型

### 3.1 安装 Paddle2ONNX

In [ ]:
!pip install paddle2onnx

### 3.2 模型转换
* 使用命令行 paddle2onnx 命令进行模型转换
* 使用时通过命令行参数指定 Paddle 推理模型的模型路径、模型文件名、参数文件名、保存文件路径和 ONNX 算子集的版本


In [ ]:
!paddle2onnx \
    --model_dir=./deploy/models/class_ResNet50_vd_ImageNet_infer \
    --model_filename=inference.pdmodel \
    --params_filename=inference.pdiparams \
    --save_file=./deploy/models/class_ResNet50_vd_ImageNet_infer/model.onnx \
    --opset_version=12

## 4. 生成配置文件


### 4.1 使用配置文件转换器
* WebAI 的体验网站中包含一个配置文件转换生成器
* 可通过这个程序快速将 PaddleClas 的配置文件转换为 WebAI.js 推理所需的配置文件
* 转换器目前仍在持续开发中，目前可能无法兼容所有 PaddleClas 的配置文件



### 4.2 手动编写配置文件
* 当然也可以手动编写一个模型配置文件，配置文件样例如下：

    ```json
    // configs.json
    {
        "Preprocess": [
            {
                "type": "Decode", // 图像解码
                "mode": "RGB" // RGB 或 BGR
            },
            {
                "type": "Resize", //  图像缩放
                "interp": 1, // 插值方式
                "keep_ratio": false, // 保持长宽比
                "limit_max": false, // 限制图片的最大尺寸
                "target_size": [300, 300] // 目标尺寸 [H, W]
            },
            /*
            {
                "type": "Crop", // 图像中心裁切
                "crop_size": [224, 224] // 目标尺寸 [H, W]
            },
            */
            {
                "type": "Normalize", // 归一化
                "is_scale": false, // 是否缩放 (img /= 255.0)
                "mean": [127.5, 127.5, 127.5], // 均值
                "std": [127.5, 127.5, 127.5] // 标准差
            },
            {
                "type": "Permute" // 转置 (HWC -> CHW)
            }
        ],
        "label_list": [
            "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", 
            "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", 
            "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
        ] // 标签列表
    }
    ```